In [1]:
import pandas as pd
import numpy as np

In [2]:
with open('../data/continents2.csv') as f:
  regions = pd.read_csv(f)
  print(regions['sub-region'].unique())

['Southern Asia' 'Northern Europe' 'Southern Europe' 'Northern Africa'
 'Polynesia' 'Sub-Saharan Africa' 'Latin America and the Caribbean' nan
 'Western Asia' 'Australia and New Zealand' 'Western Europe'
 'Eastern Europe' 'Northern America' 'South-eastern Asia' 'Eastern Asia'
 'Melanesia' 'Micronesia' 'Central Asia']


In [3]:
def clusterByRegion(Data, MeanAggregation=True):
  """
    Clusters the Dataset by region applying the sub regions found in the continents2 dataset
    
    Parameters:
        x (pd.DataFrame) the dataset to be grouped
        MeanAggregation (boolean) if true, takes the mean of Data's Features, if false, takes the sum of the features
    
    Returns:
        grouped (pd.DataFrame) a grouped dataframe 
    """
  regions = pd.read_csv("../data/continents2.csv")
  regions = regions[['alpha-3', 'sub-region']]

  merge = Data.merge(regions, how = "left", left_on = 'country_code', right_on ='alpha-3' )
  merge.drop(['country_code', 'country_name', "alpha-3"], axis=1, inplace=True)

  if MeanAggregation:
    grouped = merge.groupby(['sub-region', 'year'], as_index=False).agg('mean')
  else:
    grouped = merge.groupby(['sub-region', 'year'],as_index=False).agg('sum')
  
  return grouped
  

In [4]:
data = pd.read_csv('../data/co2_emissions.csv')
df = clusterByRegion(data, False)
print(df)

                    sub-region  year         value
0    Australia and New Zealand  1960  9.974607e+04
1    Australia and New Zealand  1961  1.023570e+05
2    Australia and New Zealand  1962  1.061230e+05
3    Australia and New Zealand  1963  1.132590e+05
4    Australia and New Zealand  1964  1.220854e+05
..                         ...   ...           ...
985             Western Europe  2015  1.417770e+06
986             Western Europe  2016  1.426180e+06
987             Western Europe  2017  1.412230e+06
988             Western Europe  2018  1.369770e+06
989             Western Europe  2019  1.309060e+06

[990 rows x 3 columns]


Group the data by `country_code`, sort each group by year, compute the anomaly using the LLR function, run permutation testing, filter for significant anomalies (p-value < 0.05), and finally sort the results by LLR in descending order.

In [5]:
def compute_llr(x):
    """
    Compute the maximum log-likelihood ratio (LLR) for a change point in a 1D array x.
    Returns the best LLR score, the index of the best change point, and the anomalous subset S*.
    
    Parameters:
        x (np.array): Array of numeric data.
    
    Returns:
        best_llr (float): Maximum improvement in fit (LLR score).
        best_t (int): Index of the best change point.
        best_subset (np.array): The anomalous subset, i.e. x[:best_t].
    """
    n = len(x)
    if n < 2:
        return None, None, None

    mean_all = np.mean(x)
    sse_all = np.sum((x - mean_all) ** 2)

    best_llr = -np.inf
    best_t = None
    best_subset = None

    for t in range(1, n):
        seg1 = x[:t]
        seg2 = x[t:]
        mean1 = np.mean(seg1)
        mean2 = np.mean(seg2)
        sse1 = np.sum((seg1 - mean1) ** 2)
        sse2 = np.sum((seg2 - mean2) ** 2)
        sse_split = sse1 + sse2
        llr = sse_all - sse_split
        if llr > best_llr:
            best_llr = llr
            best_t = t
            best_subset = x[:t]

    return best_llr, best_t, best_subset

In [6]:
results = {}

# Process the data on a per-country basis using 'country_code'
for country, group in df.groupby('sub-region'):
    group_sorted = group.sort_values('year')
    emissions = group_sorted['value'].values
    if len(emissions) < 2:
        continue

    observed_llr, best_index, anomalous_subset = compute_llr(emissions)
    
    # Extract all years corresponding to the anomalous subset
    anomalous_years = group_sorted['year'].iloc[:best_index].tolist()
    best_year = group_sorted.iloc[best_index]['year']

    num_permutations = 1000
    llr_permutations = np.zeros(num_permutations)
    for i in range(num_permutations):
        emissions_perm = np.random.permutation(emissions)
        llr_perm, _, _ = compute_llr(emissions_perm)
        llr_permutations[i] = llr_perm if llr_perm is not None else -np.inf

    p_value = np.mean(llr_permutations >= observed_llr)

    results[country] = {
        'observed_llr': observed_llr,
        'best_change_point_index': best_index,
        'best_year': best_year,
        'anomalous_years': anomalous_years,
        'p_value': p_value,
        'n_points': len(emissions)
    }

# Filter and sort results: only include countries with a p-value < 0.05, sorted by LLR descending.
filtered_sorted_results = sorted(
    [(country, res) for country, res in results.items() if res['p_value'] < 0.05],
    key=lambda item: item[1]['observed_llr'], reverse=True
)

# Display the filtered and sorted results per country
for country, res in filtered_sorted_results:
    print(f"\nCountry: {country}")
    print(f"  Observed LLR: {res['observed_llr']}")
    print(f"  Best Change Point Index: {res['best_change_point_index']} (Year: {res['best_year']})")
    print(f"  Anomalous Years: {res['anomalous_years']}")
    print(f"  p-value from permutation testing: {res['p_value']}")
    print(f"  Number of Data Points: {res['n_points']}")



Country: Eastern Asia
  Observed LLR: 658701999572440.0
  Best Change Point Index: 44 (Year: 2004)
  Anomalous Years: [1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003]
  p-value from permutation testing: 0.0
  Number of Data Points: 60

Country: Southern Asia
  Observed LLR: 43011714744201.0
  Best Change Point Index: 44 (Year: 2004)
  Anomalous Years: [1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003]
  p-value from permutation testing: 0.0
  Number of Data Points: 60

Country: Northern America
  Observed LLR: 29224947827194.402
  Best Change Point Index: 10 (Yea